In [1]:
import pandas as pd
import re

timestamps_data = pd.read_csv('data_csv/timestamps_ext.csv')
target = pd.read_csv('data_csv/first_win.csv')

target.columns = ['matchId', 'first_win']
data = timestamps_data.merge(target, on='matchId')

In [2]:
others = ['matchId', 'timeStamp', 'percentTimeStamp', "gameDuration"]

blue_team_pattern = re.compile(r'p[1-5]_(.*)')
red_team_pattern = re.compile(r'p(6|7|8|9|10)_(.*)')

blue_team_columns = {}
red_team_columns = {}

for col in data.columns:
    if blue_team_pattern.match(col):
        feature_name = blue_team_pattern.match(col).group(1)
        if feature_name not in blue_team_columns:
            blue_team_columns[feature_name] = []
        blue_team_columns[feature_name].append(col)
    elif red_team_pattern.match(col):
        feature_name = red_team_pattern.match(col).group(2)
        if feature_name not in red_team_columns:
            red_team_columns[feature_name] = []
        red_team_columns[feature_name].append(col)

def aggregate_team_columns(data, team_columns, team_prefix):
    for feature, columns in team_columns.items():
        aggregated_column = f'{team_prefix}_{feature}'
        data[aggregated_column] = data[columns].sum(axis=1)
        data.drop(columns=columns, inplace=True)
    return data

data = aggregate_team_columns(data, blue_team_columns, 'blue')
data = aggregate_team_columns(data, red_team_columns, 'red')

In [3]:
missing_values = data.isnull().sum()

print("Columns with missing values:")
print(missing_values[missing_values > 0])

# Count the number of rows with missing values
missing_rows = data.isnull().any(axis=1).sum()
total_rows = data.shape[0]

print(f"\nTotal number of rows: {total_rows}")
print(f"Number of rows with missing values: {missing_rows}")

# If the number of rows with missing values is small, drop them
if missing_rows > 0:
    data_cleaned = data.dropna()
    print(f"\nNumber of rows after dropping missing values: {data_cleaned.shape[0]}")
else:
    data_cleaned = data

# Identify and remove constant columns
constant_columns = [col for col in data_cleaned.columns if data_cleaned[col].nunique() <= 1]
print(f"\nConstant columns to be removed: {constant_columns}")

data_cleaned = data_cleaned.drop(columns=constant_columns)

X = data_cleaned.drop(columns=['first_win'] + others)
y = data_cleaned['first_win']

Columns with missing values:
Series([], dtype: int64)

Total number of rows: 363982
Number of rows with missing values: 0

Constant columns to be removed: ['blue_participantId', 'blue_championStats_abilityHaste', 'blue_championStats_armorPen', 'blue_championStats_bonusArmorPenPercent', 'blue_championStats_bonusMagicPenPercent', 'blue_championStats_cooldownReduction', 'blue_championStats_physicalVamp', 'red_participantId', 'red_championStats_abilityHaste', 'red_championStats_armorPen', 'red_championStats_bonusArmorPenPercent', 'red_championStats_bonusMagicPenPercent', 'red_championStats_cooldownReduction', 'red_championStats_physicalVamp']


In [4]:
timestamps_starting = data[data['timeStamp'] == 0]
timestamps_20p = data_cleaned.loc[data_cleaned.groupby('matchId')['percentTimeStamp'].apply(lambda x: (x - 0.2).abs().idxmin())]
timestamps_40p = data_cleaned.loc[data_cleaned.groupby('matchId')['percentTimeStamp'].apply(lambda x: (x - 0.4).abs().idxmin())]
timestamps_60p = data_cleaned.loc[data_cleaned.groupby('matchId')['percentTimeStamp'].apply(lambda x: (x - 0.6).abs().idxmin())]
timestamps_80p = data_cleaned.loc[data_cleaned.groupby('matchId')['percentTimeStamp'].apply(lambda x: (x - 0.8).abs().idxmin())]
timestamps_100p = data_cleaned.loc[data_cleaned.groupby('matchId')['percentTimeStamp'].apply(lambda x: (x - 1.0).abs().idxmin())]

In [5]:
timestamps = []
X_0 = timestamps_starting.drop(columns=['first_win']+others)
y_0 =timestamps_starting['first_win']
timestamps.append((X_0, y_0, '0'))

X_20 = timestamps_20p.drop(columns=['first_win'] + others)
y_20 = timestamps_20p['first_win']
timestamps.append((X_20, y_20, '20'))

X_40 = timestamps_40p.drop(columns=['first_win'] + others)
y_40 = timestamps_40p['first_win']
timestamps.append((X_40, y_40, '40'))

X_60 = timestamps_60p.drop(columns=['first_win'] + others)
y_60 = timestamps_60p['first_win']
timestamps.append((X_60, y_60, '60'))

X_80 = timestamps_80p.drop(columns=['first_win'] + others)
y_80 = timestamps_80p['first_win']
timestamps.append((X_80, y_80, '80'))

X_100 = timestamps_100p.drop(columns=['first_win'] + others)
y_100 = timestamps_100p['first_win']
timestamps.append((X_100, y_100, '100'))

In [6]:
correlations = X.corrwith(y).sort_values(ascending=False)

print("\nTop 10 features most positively correlated with 'first_win':")
print(correlations.head(10))

print("\nTop 10 features most negatively correlated with 'first_win':")
print(correlations.tail(10))



Top 10 features most positively correlated with 'first_win':
blue_currentGold                                  0.181166
blue_championStats_powerRegen                     0.101625
blue_championStats_health                         0.098662
blue_championStats_healthRegen                    0.065829
blue_championStats_abilityPower                   0.061727
blue_jungleMinionsKilled                          0.061165
blue_championStats_attackSpeed                    0.053465
blue_championStats_ccReduction                    0.052943
blue_totalGold                                    0.050433
blue_damageStats_physicalDamageDoneToChampions    0.048397
dtype: float64

Top 10 features most negatively correlated with 'first_win':
red_championStats_attackSpeed                   -0.059651
red_championStats_ccReduction                   -0.060745
red_damageStats_physicalDamageDoneToChampions   -0.061382
red_totalGold                                   -0.061998
red_championStats_healthRegen          

In [7]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def evaluate_model(model, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)

    print(f"Model: {model.__class__.__name__}")
    print(f"Accuracy: {accuracy:.4f}")

models = [
    LogisticRegression(random_state=42, n_jobs=-1),
    DecisionTreeClassifier(random_state=42),
    RandomForestClassifier(random_state=42, n_jobs=-1),
    HistGradientBoostingClassifier(random_state=42)
]

In [8]:
from sklearn.model_selection import cross_val_score

def cross_val_model(model, X, y):
    # Perform cross-validation and get accuracy scores
    scores = cross_val_score(model, X, y, cv=5, n_jobs=-1, scoring='accuracy')
    # Print the model's class name
    print(f"Model: {model.__class__.__name__}")
    # Print all cross-validation scores
    print(f"Cross-validation scores (Accuracy): {scores}")
    # Print the mean cross-validation score
    print(f"Mean cross-validation score (Accuracy): {scores.mean():.4f}\n")

In [9]:
def test_timestamp(timestamp):
    X, y, name = timestamp
    print('timestamp at {} percent'.format(name))
    for model in models:
        evaluate_model(model, X, y)
    
    # uncomment to also test with cross_validation - takes more time and results are similar so I left it commented out for now
    # for model in models:
    #     cross_val_model(model, X, y)

In [10]:
test_timestamp(timestamps[0])

timestamp at 0 percent
Model: LogisticRegression
Accuracy: 0.5117
Model: DecisionTreeClassifier
Accuracy: 0.5036
Model: RandomForestClassifier
Accuracy: 0.5065
Model: HistGradientBoostingClassifier
Accuracy: 0.5180


In [11]:
test_timestamp(timestamps[1])

timestamp at 20 percent
Model: LogisticRegression
Accuracy: 0.6226
Model: DecisionTreeClassifier
Accuracy: 0.5482
Model: RandomForestClassifier
Accuracy: 0.6100
Model: HistGradientBoostingClassifier
Accuracy: 0.6189


In [12]:
test_timestamp(timestamps[2])

timestamp at 40 percent
Model: LogisticRegression
Accuracy: 0.6953
Model: DecisionTreeClassifier
Accuracy: 0.6257
Model: RandomForestClassifier
Accuracy: 0.6836
Model: HistGradientBoostingClassifier
Accuracy: 0.6895


In [13]:
test_timestamp(timestamps[3])

timestamp at 60 percent
Model: LogisticRegression
Accuracy: 0.7633
Model: DecisionTreeClassifier
Accuracy: 0.6892
Model: RandomForestClassifier
Accuracy: 0.7564
Model: HistGradientBoostingClassifier
Accuracy: 0.7596


In [14]:
test_timestamp(timestamps[4])

timestamp at 80 percent
Model: LogisticRegression
Accuracy: 0.8507
Model: DecisionTreeClassifier
Accuracy: 0.7889
Model: RandomForestClassifier
Accuracy: 0.8478
Model: HistGradientBoostingClassifier
Accuracy: 0.8496


In [15]:
test_timestamp(timestamps[5])

timestamp at 100 percent
Model: LogisticRegression
Accuracy: 0.9722
Model: DecisionTreeClassifier
Accuracy: 0.9306
Model: RandomForestClassifier
Accuracy: 0.9758
Model: HistGradientBoostingClassifier
Accuracy: 0.9802
